<a href="https://colab.research.google.com/github/btlgs2000/dl_intro/blob/master/model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pr0ova